# 从解方程开始

In [3]:
from z3 import *
set_option(html_mode=False)
x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


`Int('x')` 函数在 Z3 中创建了一个名为 `x` 的整数变量。`solve` 函数用于解决一组约束条件。上面的示例使用了两个变量 `x` 和 `y`，以及三个约束条件。与 Python 类似，Z3Py 使用 `=` 进行赋值操作，使用 `<`、`<=`、`>`、`>=`、`==` 和 `!=` 进行比较操作。在上面的示例中，表达式 `x + 2*y == 7` 是一个 Z3 的约束条件。Z3 能够解决和计算公式。

下面的示例展示了如何使用 Z3 的公式/表达式简化器。

In [4]:
x = Int('x')
y = Int('y')
print (simplify(x + y + 2*x + 3))
print (simplify(x < y + x + 2))
print (simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)))

3 + 3*x + y
Not(y <= -2)
And(x >= 2, 2*x**2 + y**2 >= 3)


In [5]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 > 3, x**3 + y < 5)

[y = 2, x = 1/8]


`Real('x')` 过程创建了实数变量 `x`。Z3Py 可以表示任意大的整数、有理数（就像上面的示例中一样）以及无理代数数。无理代数数是具有整数系数的多项式的根。在内部，Z3精确地表示所有这些数。为了便于阅读结果，无理数以十进制表示方式显示出来。

In [6]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

set_option(precision=3)

[y = -1.1885280594?, x = 1.2599210498?]


`set_option` 过程用于配置Z3环境。它用于设置全局配置选项，例如结果的显示方式。选项 `set_option(precision=30)` 设置在显示结果时使用的小数位数。1.2599210498? 中的问号表示输出被截断。

以下示例演示了一个常见的错误。表达式 `3/2` 是Python整数，而不是Z3有理数。示例还展示了在Z3Py中创建有理数的不同方法。`Q(num, den)` 过程创建了一个Z3有理数，其中 `num` 是分子，`den` 是分母。`RealVal(1)` 创建了一个表示数字1的Z3实数。


In [7]:
print (1/3)
print (RealVal(1)/3)
print (Q(1,3))

x = Real('x')
print (x + 1/3)
print (x + Q(1,3))
print (x + "1/3")
print (x + 0.25)

0.3333333333333333
1/3
1/3
x + 3333333333333333/10000000000000000
x + 1/3
x + 1/3
x + 1/4


# 求解器

Z3提供了不同的求解器。在前面的示例中使用的`solve`命令是使用Z3求解器API实现的。该实现可以在Z3发行版中的`z3.py`文件中找到。以下示例演示了基本的Solver API。


In [9]:
x = Int('x')
y = Int('y')

s = Solver()
print (s)

[]


In [11]:
s.add(x > 10, y == x + 2)
print (s)
print ("Solving constraints in the solver s ...")
print (s.check())

[x > 10, y == x + 2, x > 10, y == x + 2]
Solving constraints in the solver s ...
sat


In [12]:
print ("Create a new scope...")

# Create a backtracking point.
s.push()
s.add(y < 11)
print (s)
print ("Solving updated set of constraints...")
print (s.check())

print ("Restoring state...")

# Restore a backtracking point.
s.pop()
print (s)
print ("Solving restored set of constraints...")
print (s.check())

Create a new scope...
[x > 10, y == x + 2, x > 10, y == x + 2, y < 11]
Solving updated set of constraints...
unsat
Restoring state...
[x > 10, y == x + 2, x > 10, y == x + 2]
Solving restored set of constraints...
sat


`Solver()` 命令创建一个通用求解器。可以使用 `add` 方法添加约束条件。我们说这些约束条件已经被在求解器中断言。`check()` 方法用于解决断言的约束条件。如果找到了解，则结果为 `sat`（可满足）。如果不存在解，则结果为 `unsat`（不可满足）。我们也可以说断言的约束条件系统是不可行的。最后，求解器可能无法解决一组约束条件，会返回 `unknown`。

在某些应用中，我们希望探索共享多个约束条件的几个类似问题。我们可以使用 `push` 和 `pop` 命令来实现。每个求解器都维护一个断言的堆栈。`push` 命令通过保存当前堆栈大小来创建一个新的作用域。`pop` 命令会移除它和匹配的 `push` 之间执行的任何断言。`check` 方法始终在求解器断言堆栈的内容上操作。

以下示例展示了一个Z3无法解决的示例。在这种情况下，求解器返回 `unknown`。请注意，Z3可以解决非线性多项式约束，但是 `2**x` 不是多项式。


In [13]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print (s.check())

unknown


`check` 命令在Z3找到断言的约束条件集的解时返回 `sat`。我们称Z3满足了这组约束条件。我们说这个解是断言的约束条件集的一个模型。模型是一种解释，使每个断言的约束条件都为真。以下示例展示了检查模型的基本方法。

In [14]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print (s.check())

m = s.model()
print ("x = %s" % m[x])

print ("traversing model...")
for d in m.decls():
    print ("%s = %s" % (d.name(), m[d]))

sat
x = 3/2
traversing model...
y = 2
x = 3/2
z = 0


# 算术

Z3支持实数和整数变量，它们可以在单个问题中混合使用。与大多数编程语言一样，Z3Py在需要时会自动添加强制转换，将整数表达式转换为实数表达式。以下示例演示了声明整数和实数变量的不同方法。

In [15]:
a, b, c = Ints('a b c')
d, e = Reals('d e')
solve(a > b + 2,
      a == 2*c + 10,
      c + b <= 1000,
      d >= e)

x, y = Reals('x y')
# Put expression in sum-of-monomials form
t = simplify((x + y)**3, som=True)
print (t)
# Use power operator
t = simplify(t, mul_to_power=True)
print (t)

[d = 0, c = 0, b = 0, e = 0, a = 10]
x*x*x + 3*x*x*y + 3*x*y*y + y*y*y
x**3 + 3*x**2*y + 3*x*y**2 + y**3


# (数学意义上的)函数

In [19]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
solve(f(f(x)) == x, f(x) == y)

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


In [18]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print (s.check())
m = s.model()
print ("f(f(x)) =", m.evaluate(f(f(x))))
print ("f(x)    =", m.evaluate(f(x)))

sat
f(f(x)) = 0
f(x)    = 1


# 选择的问题

1. 在高中，学生学习了运动学方程。这些方程描述了位移（$d$）、时间（$t$）、加速度（$a$）、初始速度（$v_i$）和最终速度（$v_f$）之间的数学关系。在Z3Py中，我们可以将这些方程写成：


In [20]:
d, a, t, v_i, v_f = Reals('d a t v__i v__f')

equations = [
   d == v_i * t + (a*t**2)/2,
   v_f == v_i + a*t,
]
print ("Kinematic equations:")
print (equations)

# Given v_i, v_f and a, find d
problem = [
    v_i == 30,
    v_f == 0,
    a   == -8
]
print ("Problem:")
print (problem)

print ("Solution:")
solve(equations + problem)

Kinematic equations:
[d == v__i*t + (a*t**2)/2, v__f == v__i + a*t]
Problem:
[v__i == 30, v__f == 0, a == -8]
Solution:
[t = 15/4, v__f = 0, v__i = 30, d = 225/4, a = -8]


2. 考虑以下的问题。花费正好100元并购买正好100只动物。狗的价格是15元，猫的价格是1元，而老鼠每只价格是0.25元。至少每种购买一只动物。每种动物分别应该购买多少只？

In [21]:
# Create 3 integer variables
dog, cat, mouse = Ints('dog cat mouse')
solve(dog >= 1,   # at least one dog
      cat >= 1,   # at least one cat
      mouse >= 1, # at least one mouse
      # we want to buy 100 animals
      dog + cat + mouse == 100,
      # We have 100 dollars (10000 cents):
      #   dogs cost 15 dollars (1500 cents), 
      #   cats cost 1 dollar (100 cents), and 
      #   mice cost 25 cents 
      1500 * dog + 100 * cat + 25 * mouse == 10000)

[mouse = 56, cat = 41, dog = 3]


3. 数独游戏


In [22]:
# 9x9 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]

# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, we use '0' for empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print ("failed to solve")


[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]
